# 1,2 Setup - Imports, Scraping, getting raw dataframe

In [219]:
import requests
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'folium'

In [164]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(link)
html_code = response.text

In [165]:
#ts - table start
#te - table end
ts_str = "<table class=\"wikitable sortable\">"
te_str = "</table"
ts_idx = html_code.find(ts_str) 
te_idx = html_code.find(te_str) + len(te_str)

In [166]:
table_html = code[ts_idx:te_idx]

In [167]:
df_list = pd.read_html(table_html,header=0)
df_raw = df_list[0]

# 3 - To create the above dataframe:

#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [168]:
df_wbna = df_raw[df_raw.Borough != "Not assigned"] #wbna = without borought not assigned

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [169]:
for i, row in df_wbna.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood'] = row['Borough']

#### More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma

In [170]:
df_gbp = df_wbna.groupby('Postcode') #gbp - group by postcode

In [171]:
se_mg = df_gbp['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)) #se_mg - series merged

In [172]:
df_mg = se_mg.to_frame() #df_mg = df merged

**Insert boroughs to df_mg**

In [173]:
df_mg["Borough"] = ""

In [174]:
df_mg.reset_index(level=0, inplace=True)

In [175]:
for i, row in df_mg.iterrows():
    dft = df_raw.loc[df_raw['Postcode'] == row['Postcode']]
    row['Borough'] = dft.iloc[0,1]

#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [176]:
#cosmetic adjustments
df_mg = df_mg[['Postcode', 'Borough', 'Neighbourhood']]
df_mg.rename(index=str,columns={"Postcode":"PostalCode","Neighbourhood":"Neighborhood"},inplace=True)

In [214]:
df_mg.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [215]:
df_mg.shape

(103, 3)

# Map Stuff

### Setup

In [221]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.3.0               |        py36_1001         533 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.7 MB

The following NEW packages will be INSTALLED:

    altair:  2.3.0-py36_1001   conda-forge
    branca:  0.3.1-py_0        conda-forge
    folium:  0.5.0-py_0        conda-forge
    vincent: 0.4.

In [222]:
import folium # map rendering library

#### Use the Geocoder package or the csv file to create the following dataframe:

In [190]:
df_ll = pd.read_csv("http://cocl.us/Geospatial_data")

In [191]:
df_ll.rename(index=str,columns={'Postal Code':'PostalCode'},inplace=True)

In [240]:
df = pd.merge(left=df_mg, right=df_ll ,on='PostalCode')

In [241]:
df = df[df.Borough.str.contains("Toronto")]

In [242]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [243]:
map_tr = folium.Map(location=[43.806686, -79.194353], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tr)  
    
map_tr